In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import json
from tqdm.notebook import tqdm

def split_csv(file_path, train_ratio=0.9, random_state=43):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Split the data into train and validation sets
    train_data, val_data = train_test_split(df, test_size=(1-train_ratio), random_state=random_state)
    
    # Generate new file names
    base_name = file_path.rsplit('.', 1)[0]
    train_file = f"split_train.csv"
    val_file = f"split_val.csv"
    
    # Save the split datasets
    train_data.to_csv(train_file, index=False)
    val_data.to_csv(val_file, index=False)
    
    print(f"Training set saved to: {train_file}")
    print(f"Validation set saved to: {val_file}")
    
    return train_file, val_file

# Usage example
file_path = "./COMP90086_2024_Project_train/train.csv"
train_file, val_file = split_csv(file_path)

Training set saved to: split_train.csv
Validation set saved to: split_val.csv


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import json

def calculate_image_stats(csv_file, csv_name, img_dir, use_quantized):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Initialize variables for mean and std calculation
    sum_means = np.zeros(3)
    sum_stds = np.zeros(3)
    count = 0

    # Iterate through all images
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
        img_name = str(row[0])
        if use_quantized:
            img_path = os.path.join(img_dir, f"quantized/{img_name}_quantized.jpg")
        else:
            img_path = os.path.join(img_dir, f"{img_name}_original.jpg")
        
        # Skip augmented images
        if "_flipped" in img_path or "_zoomed" in img_path:
            continue

        # Read and process the image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Warning: Could not read image {img_path}")
            continue
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Calculate mean and std for each channel
        means = np.mean(image, axis=(0, 1))
        stds = np.std(image, axis=(0, 1))
        
        sum_means += means
        sum_stds += stds
        count += 1

    # Calculate final mean and std
    final_mean = sum_means / count
    final_std = sum_stds / count

    stats = {
        "mean": final_mean.tolist(),
        "std": final_std.tolist()
    }

    quant = "_quantized" if use_quantized else ""
    output_file = f"dataset_stats/{csv_name}{quant}.json"
    with open(output_file, "w") as f:
        json.dump(stats, f, indent=4)

    print(f"Image statistics saved to {output_file}")
    print(f"Mean: {final_mean}")
    print(f"Std: {final_std}")

img_dir = './preprocessed_images/train'
main_train = './COMP90086_2024_Project_train/train.csv'
split_train = 'split_train.csv'

for quant in [True, False]:
    calculate_image_stats(main_train, "full", img_dir, quant)
    calculate_image_stats(split_train, "split", img_dir, quant)


Processing images:   0%|          | 0/7680 [00:00<?, ?it/s]/tmp/ipykernel_272164/354567196.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 7680/7680 [01:39<00:00, 77.22it/s] 


Image statistics saved to dataset_stats/full_quantized.json
Mean: [118.6946178  111.99958266 103.1688242 ]
Std: [68.48892217 57.7008792  47.95330727]


Processing images:   0%|          | 0/6912 [00:00<?, ?it/s]/tmp/ipykernel_272164/354567196.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 6912/6912 [00:58<00:00, 119.13it/s]


Image statistics saved to dataset_stats/split_quantized.json
Mean: [118.8203705  112.10561837 103.24939132]
Std: [68.55053308 57.7906873  48.00791136]


Processing images:   0%|          | 0/7680 [00:00<?, ?it/s]/tmp/ipykernel_272164/354567196.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 7680/7680 [00:42<00:00, 179.83it/s]


Image statistics saved to dataset_stats/full.json
Mean: [119.21526968 112.51073973 103.68285506]
Std: [69.10808986 58.16075209 48.56616013]


Processing images:   0%|          | 0/6912 [00:00<?, ?it/s]/tmp/ipykernel_272164/354567196.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_name = str(row[0])
Processing images: 100%|██████████| 6912/6912 [00:47<00:00, 145.23it/s]

Image statistics saved to dataset_stats/split.json
Mean: [119.34120849 112.61692033 103.76272418]
Std: [69.17001857 58.25038132 48.62095091]
